In [25]:
import sys
sys.path.append('/home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *
from pythreejs import *
from time import sleep
from IPython.display import display
from pythreejs import *

In [26]:
# Create System
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

pivot = mss.add(Fix((0, 0, 0)))

m1 = mss.add(Mass(1.0, (1.0, 0.0, 0.0)))
m2 = mss.add(Mass(1.0, (2.0, 0.0, 0.0)))

ndof = 3 * len(mss.masses)

mss.addConstraint(DistanceConstraint3d(ndof, pivot, m1, 1.0, mss))

mss.addConstraint(DistanceConstraint3d(ndof, m1, m2, 1.0, mss))

In [27]:
#Visual Objects
vis_masses = [Mesh(SphereBufferGeometry(0.2), MeshStandardMaterial(color='red'), position=m.pos) for m in mss.masses]
vis_fixes  = [Mesh(SphereBufferGeometry(0.2), MeshStandardMaterial(color='blue'), position=f.pos) for f in mss.fixes]

#Helper to get Line Positions
def get_segments():
    p0 = mss.fixes[0].pos
    p1 = mss.masses[0].pos
    p2 = mss.masses[1].pos
    return [[p0, p1], [p1, p2]]

#Create Lines
lines = LineSegments2(
    LineSegmentsGeometry(positions=get_segments()),
    LineMaterial(linewidth=3, color='black')
)

#Scene & Camera
camera = PerspectiveCamera(position=[0, -5, 0], aspect=800/400)
scene = Scene(children=[*vis_masses, *vis_fixes, lines, camera, DirectionalLight(position=[0, 10, 10]), AmbientLight()])

renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=800, height=400)
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=2.0, position=(0.0, -5.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0,…

In [28]:
# Run Simulation
for i in range(1000):
    # Physics Step
    mss.simulate(0.01, 50)
    
    # Update Spheres
    for m, mvis in zip(mss.masses, vis_masses):
        mvis.position = tuple(m.pos)
        
    # Update Lines
    lines.geometry = LineSegmentsGeometry(positions=get_segments())
    
    sleep(0.01)